In [1]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propogate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import ResolverNode
from tfx.components import Evaluator
from tfx.components import Pusher

from tfx.dsl.experimental import latest_blessed_model_resolver

from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.utils.dsl_utils import external_input

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

/home/kaimo/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [2]:
## Set up logging
absl.logging.set_verbosity(absl.logging.INFO)

In [3]:
## Download data
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, 'data.csv')
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-dataupi3efus/data.csv', <http.client.HTTPMessage at 0x7fa2d4acff10>)

In [4]:
## Create interacitveContext
context = InteractiveContext()

## Run TFX components interactively

### ExampleGen

In [5]:
example_gen = CsvExampleGen(input=external_input(_data_root))
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /tmp/tfx-dataupi3efus/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/CsvExampleGen/examples/1, id: 2)]
        ))

### StatisticsGen

In [6]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/StatisticsGen/statistics/2/eval.
/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/StatisticsGen/statistics/2, id: 3)]
        ))

In [7]:
%%skip_for_export
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.


### SchemaGen

In [8]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.
INFO:absl:Schema written to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/SchemaGen/schema/3, id: 4)]
        ))

In [9]:
%%skip_for_export
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,optional,single,'company'
'dropoff_census_tract',INT,optional,single,-
'dropoff_community_area',INT,optional,single,-
'dropoff_latitude',FLOAT,optional,single,-
'dropoff_longitude',FLOAT,optional,single,-
'fare',FLOAT,required,single,-
'payment_type',STRING,required,single,'payment_type'
'pickup_census_tract',INT,optional,single,-
'pickup_community_area',INT,required,single,-


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '1085 - 72312 N and W Cab Co', '2092 - 61288 Sbeih company', '2192 - 73487 Zeymane Corp', '2192 - Zeymane Corp', '2733 - 74600 Benny Jona', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3011 - 66308 JBL Cab Inc.', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3385 - 23210 Eman Cab', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - 57856 Ilie Malec', '3897 - Ilie Malec', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '5129 - Mengisti Taxi', '585 - Valley Cab Co', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5874 - Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '5997 - AW Services Inc.', '6057 - 24657 Richard Addo', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', '6742 - 83735 Tasha ride inc', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Unknown'"


This cell will be skipped during export to pipeline.


### ExampleValidator

In [10]:
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/ExampleValidator/anomalies/4.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/ExampleValidator/anomalies/4, id: 5)]
        ))

In [11]:
%%skip_for_export
context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'payment_type',Unexpected string values,Examples contain values missing from the schema: Prcard (<1%).
'pickup_census_tract',Column dropped,The feature was not present in any examples.
'company',Unexpected string values,"Examples contain values missing from the schema: 3094 - 24059 G.L.B. Cab Co (<1%), 3319 - CD Cab Co (<1%), 4053 - 40193 Adwar H. Nikola (<1%), 4197 - Royal Star (<1%), 5006 - Salifu Bawa (<1%), 5724 - KYVI Cab Inc (<1%), 585 - 88805 Valley Cab Co (<1%), 6743 - Luhak Corp (<1%)."


This cell will be skipped during export to pipeline.


### Transform

In [12]:
_taxi_constants_module_file = 'taxi_constants.py'

In [13]:
%%skip_for_export
%%writefile {_taxi_constants_module_file}

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]
CATEGORICAL_FEATURE_KEYS = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

DENSE_FLOAT_FEATURE_KEYS = ['trip_miles', 'fare', 'trip_seconds']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10
BUCKET_FEATURE_KEYS = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000
# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10
VOCAB_FEATURE_KEYS = [
    'payment_type',
    'company',
]

# Keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'

def transformed_name(key):
    return key + '_xf'

Overwriting taxi_constants.py
This cell will be skipped during export to pipeline.


In [14]:
_taxi_transform_module_file = 'taxi_transform.py'

In [23]:
%%skip_for_export
%%writefile {_taxi_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import taxi_constants

_DENSE_FLOAT_FEATURE_KEYS = taxi_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = taxi_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = taxi_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = taxi_constants.CATEGORICAL_FEATURE_KEYS
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY
_transformed_name = taxi_constants.transformed_name

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    for key in _DENSE_FLOAT_FEATURE_KEYS:
        # Preserve this feature as a dense float, setting nan's to the mean.
        outputs[_transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))

    for key in _VOCAB_FEATURE_KEYS:
        # Build a vocabulary for this feature.
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key]),
            top_k=_VOCAB_SIZE,
            num_oov_buckets=_OOV_SIZE
        )

    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            _fill_in_missing(inputs[key]),
            num_buckets=_FEATURE_BUCKET_COUNT,
            always_return_num_quantiles=False
        )
    
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])
        
    # Was this passenger a big tipper? - our classification goal
    taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
    tips = _fill_in_missing(inputs[_LABEL_KEY])
    outputs[_transformed_name(_LABEL_KEY)] = tf.where(
        tf.math.is_nan(taxi_fare),
        tf.cast(tf.zeros_like(taxi_fare), tf.int64),
        tf.cast(tf.greater(tips, taxi_fare*tf.constant(0.2)), tf.int64)
    )
    
    return outputs

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
        x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
          in the second dimension.
    Returns:
        A rank 1 tensor where missing values of `x` have been filled in.
    """
    default_value = "" if x.dtype == tf.string else 0
    return tf.squeeze(
        tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]), default_value=default_value
        ),
        axis=1
    )

Overwriting taxi_transform.py
This cell will be skipped during export to pipeline.


In [24]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_transform_module_file)
)
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Using 1 process(es) for Beam pipeline execution.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Transform/transform_graph/9/.temp_path/tftransform_tmp/42c346f5f19f45f181478893f6ff3ffe/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 9
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Transform/transform_graph/9, id: 14)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Transform/transformed_examples/9, id: 15)]
        ))

### Trainer

In [25]:
_taxi_trainer_module_file = 'taxi_trainer.py'

In [67]:
%%skip_for_export
%%writefile {_taxi_trainer_module_file}

import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

import taxi_constants

_DENSE_FLOAT_FEATURE_KEYS = taxi_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = taxi_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_BUCKET_FEATURE_KEYS= taxi_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_CATEGORICAL_FEATURE_KEYS = taxi_constants.CATEGORICAL_FEATURE_KEYS
_MAX_CATEGORICAL_FEATURE_VALUES = taxi_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = taxi_constants.LABEL_KEY
_transformed_name = taxi_constants.transformed_name

def _transformed_names(keys):
    return [_transformed_name(key) for key in keys]


# Tf.Transform considers these features as "raw"
def _get_raw_feature_spec(schema):
    return schema_utils.schema_as_feature_spec(schema).feature_spec


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def _build_estimator(config, hidden_units=None, warm_start_from=None):
    """Build an estimator for predicting the tipping behavior of taxi riders.
    Args:
        config: tf.estimator.RunConfig defining the runtime environment for the
          estimator (including model_dir).
        hidden_units: [int], the layer sizes of the DNN (input layer first)
        warm_start_from: Optional directory to warm start from.
    Returns:
        A dict of the following:
          - estimator: The estimator that will be used for training and eval.
          - train_spec: Spec for training.
          - eval_spec: Spec for eval.
          - eval_input_receiver_fn: Input function for eval.
    """
    real_valued_columns = [
        tf.feature_column.numeric_column(key, shape=())
        for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
    ]
    categorical_columns = [
        tf.feature_column.categorical_column_with_identity(
            key,
            num_buckets=_VOCAB_SIZE+_OOV_SIZE,
            default_value=0
        )
        for key in _transformed_names(_VOCAB_FEATURE_KEYS)
    ]
    categorical_columns += [
        tf.feature_column.categorical_column_with_identity(
            key,
            num_buckets=_FEATURE_BUCKET_COUNT,
            default_value=0
        )
        for key in _transformed_names(_BUCKET_FEATURE_KEYS)
    ]
    categorical_columns += [
        tf.feature_column.categorical_column_with_identity(
            key,
            num_buckets=num_buckets,
            default_value=0
        )
        for key, num_buckets in zip(
            _transformed_names(_CATEGORICAL_FEATURE_KEYS),
            _MAX_CATEGORICAL_FEATURE_VALUES
        )
    ]
    return tf.estimator.DNNLinearCombinedClassifier(
        config=config,
        linear_feature_columns=categorical_columns,
        dnn_feature_columns=real_valued_columns,
        dnn_hidden_units=hidden_units or [100, 70, 50, 25],
        warm_start_from=warm_start_from
    )

def _example_serving_receiver_fn(tf_transform_graph, schema):
    """Build the serving in inputs.
    Args:
        tf_transform_graph: A TFTransformOutput.
        schema: The schema of the input data.
    Returns:
        Tensorflow graph which parses examples, applying tf-transform to them.
    """
    raw_feature_spec = _get_raw_feature_spec(schema)
    raw_feature_spec.pop(_LABEL_KEY)
    
    raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(raw_feature_spec, default_batch_size=None)
    serving_input_receiver = raw_input_fn()
    
    transformed_features = tf_transform_graph.transform_raw_features(serving_input_receiver.features)
    
    return tf.estimator.export.ServingInputReceiver(transformed_features, serving_input_receiver.receiver_tensors)

def _eval_input_receiver_fn(tf_transform_graph, schema):
    """Build everything needed for the tf-model-analysis to run the model.
    Args:
        tf_transform_graph: A TFTransformOutput.
        schema: The schema of the input data.
    Returns:
        EvalInputReceiver function, which contains:
          - Tensorflow graph which parses raw untransformed features, applies the 
            tf-transform preprocessing operators.
          - Set of raw, untransformed features.
          - Label against which predictions will be compared.
    """
    # Notice that the inputs are raw features, not transformed features here.
    raw_feature_spec = _get_raw_feature_spec(schema)
    
    serialized_tf_example = tf.compat.v1.placeholder(
        dtype=tf.string,
        shape=[None,],
        name='input_example_tensor'
    )
    
    # Add a parse_example operator to the tensorflow graph, which will parse
    # raw, untransformed, tf examples.
    features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    
    # Now that we have our raw examples, process them through the tf-transform
    # function computed during the preprocessing step.
    transformed_features = tf_transform_graph.transform_raw_features(features)
    
    # The key name MUST be 'examples'
    receiver_tensors = {'examples': serialized_tf_example}
    
    # NOTE: Model is driven by transformed features (since training works on the
    # materialized output of TFT, but slicing will happen on raw features)
    features.update(transformed_features)
    
    return tfma.export.EvalInputReceiver(
        features=features,
        receiver_tensors=receiver_tensors,
        labels=transformed_features[_transformed_name(_LABEL_KEY)]
    )

def _input_fn(filenames, tf_transform_graph, batch_size=200):
    """Generate features and labels for training or evaluation.
    Args:
        filenames: [str] list of CSV files to read data from.
        tf_transform_graph, A TFTransformOutput.
        batch_size: int First dimension size of the Tensors returned by input_fn.
    Returns:
        A (features, indices) tuple where features is a dictionary of Tensors,
          and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = tf_transform_graph.transformed_feature_spec().copy()
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        filenames,
        batch_size,
        transformed_feature_spec,
        reader=_gzip_reader_fn
    )
    
    transformed_features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    # We pop the label because we do not want to use it as a feature while we're training
    return transformed_features, transformed_features.pop(_transformed_name(_LABEL_KEY))

# TFX will call this function
def trainer_fn(trainer_fn_args, schema):
    """Build the estimator using the high level API.
    Args:
        trainer_fn_args: Holds args used to train the model as name/value pairs.
        schema: Holds the schema of the training examples.
    Returns:
        A dict of the following:
          - estimator: The estimator that will be used for traning and eval.
          - train_spec: Spec for training.
          - eval_spec: Spec for eval.
          - eval_input_receiver_fn: Input function for eval.
    """
    # Number of nodes in the first layer of the DNN
    first_dnn_layer_size = 100
    num_dnn_layers = 4
    dnn_decay_factor = 0.7
    
    train_batch_size = 40
    eval_batch_size = 40
    
    tf_transform_graph = tft.TFTransformOutput(trainer_fn_args.transform_output)
    
    train_input_fn = lambda: _input_fn(
        filenames=trainer_fn_args.train_files,
        tf_transform_graph=tf_transform_graph,
        batch_size=train_batch_size
    )
    
    eval_input_fn = lambda: _input_fn(
        filenames=trainer_fn_args.eval_files,
        tf_transform_graph=tf_transform_graph,
        batch_size=eval_batch_size
    )
    
    train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn,
        max_steps=trainer_fn_args.train_steps
    )
    
    serving_receiver_fn = lambda: _example_serving_receiver_fn(tf_transform_graph, schema)
    
    exporter = tf.estimator.FinalExporter('chicago-taxi', serving_receiver_fn)
    eval_spec = tf.estimator.EvalSpec(
        input_fn=eval_input_fn,
        steps=trainer_fn_args.eval_steps,
        exporters=[exporter],
        name='chicago-taxi-eval'
    )
    
    run_config = tf.estimator.RunConfig(save_checkpoints_steps=999, keep_checkpoint_max=1)
    run_config = run_config.replace(model_dir=trainer_fn_args.serving_model_dir)
    
    estimator = _build_estimator(
        config=run_config,
        hidden_units=[max(2, int(first_dnn_layer_size * dnn_decay_factor**i)) for i in range(num_dnn_layers)],
        warm_start_from=trainer_fn_args.base_model
    )
    
    # Create an input receiver for TFMA processing
    receiver_fn = lambda: _eval_input_receiver_fn(tf_transform_graph, schema)
    
    return {
        'estimator': estimator,
        'train_spec': train_spec,
        'eval_spec': eval_spec,
        'eval_input_receiver_fn': receiver_fn
    }

Overwriting taxi_trainer.py
This cell will be skipped during export to pipeline.


In [68]:
trainer = Trainer(
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_trainer_module_file),
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000)
)
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/serving_model_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 999, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:absl:Training model.


INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 999 or save_checkpoints_secs None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/serving_model_dir/model.ckpt.
INFO:tensorflow:loss = 0.6900983, step = 0
INFO:tensorflow:global_step/sec: 64.6318
INFO:tensorflow:loss = 0.63924897, step = 100 (1.548 sec)
INFO:tensorflow:global_step/sec: 98.1947
INFO:tensorflow:loss = 0.5147902, step = 200 (1.020 sec)
INFO:tensorflow:global_step/s

INFO:tensorflow:global_step/sec: 93.4587
INFO:tensorflow:loss = 0.35701716, step = 5100 (1.070 sec)
INFO:tensorflow:global_step/sec: 94.6938
INFO:tensorflow:loss = 0.465964, step = 5200 (1.056 sec)
INFO:tensorflow:global_step/sec: 93.7685
INFO:tensorflow:loss = 0.31716508, step = 5300 (1.065 sec)
INFO:tensorflow:global_step/sec: 93.7577
INFO:tensorflow:loss = 0.38606277, step = 5400 (1.067 sec)
INFO:tensorflow:global_step/sec: 93.5911
INFO:tensorflow:loss = 0.3175949, step = 5500 (1.069 sec)
INFO:tensorflow:global_step/sec: 93.6017
INFO:tensorflow:loss = 0.3369127, step = 5600 (1.067 sec)
INFO:tensorflow:global_step/sec: 92.6399
INFO:tensorflow:loss = 0.36339244, step = 5700 (1.081 sec)
INFO:tensorflow:global_step/sec: 93.9771
INFO:tensorflow:loss = 0.47656727, step = 5800 (1.064 sec)
INFO:tensorflow:global_step/sec: 94.3085
INFO:tensorflow:loss = 0.31874675, step = 5900 (1.060 sec)
INFO:tensorflow:Saving checkpoints for 5994 into /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2e

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/serving_model_dir/model.ckpt-10000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/serving_model_dir/export/chicago-taxi/temp-b'1589427087'/assets
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/serving_model_dir/export/chicago-taxi/temp-b'1589427087'/saved_model.pb
INFO:tensorfl

INFO:absl:Training complete.  Model written to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/serving_model_dir
INFO:absl:Exporting eval_savedmodel for TFMA.


value: "\n\013\n\tConst_7:0\022-vocab_compute_and_apply_vocabulary_vocabulary"

value: "\n\013\n\tConst_9:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: ['eval']
INFO:tensorflow:Restoring parameters from /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/serving_model_dir/model.ckpt-10000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/eval_model_dir/temp-b'1589427088'/assets
INFO:tensorflow:

INFO:absl:Exported eval_savedmodel to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/eval_model_dir.
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 14
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14, id: 20)]
        ))

### Evaluator

In [69]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # Using signature 'eval' implies the use of an EvalSavedModel. To use
        # a serving model remove the signature to defaults to 'serving_default'
        # and add a label_key.
        tfma.ModelSpec(signature_name='eval')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            metrics=[tfma.MetricConfig(class_name='ExampleCount')],
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            thresholds={
                'accuracy': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': -1e-10}
                    )
                )
            }
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(feature_keys=['trip_start_hour'])
    ]
)

In [70]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = ResolverNode(
    instance_name='latest_blessed_model_resolver',
    resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    #baseline_model=model_resolver.outputs['model'],
    # Change threshold will be ignored if there is no baseline (first run).
    eval_config=eval_config)
context.run(evaluator)

INFO:absl:Running driver for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  signature_name: "eval"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "trip_start_hour"
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
    threshold {
    }
  }
  thresholds {
    key: "accuracy"
    value {
      value_threshold {
        lower_bound {
          value: 0.5
        }
      }
    }
  }
}

INFO:absl:Using /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/eval_model_dir/1

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Trainer/model/14/eval_model_dir/1589427088/variables/variables
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.


INFO:absl:Evaluation complete. Results written to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Evaluator/evaluation/16.
INFO:absl:Checking validation results.
INFO:absl:Blessing result True written to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Evaluator/blessing/16.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 16
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Evaluator/evaluation/16, id: 21)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Evaluator/blessing/16, id: 22)]
        ))

### Pusher

In [72]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/taxi_simple')
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(base_directory=_serving_model_dir)
    )
)
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model pushing.
INFO:absl:Model version is 1589427087
INFO:absl:Model written to /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Pusher/pushed_model/17.
INFO:absl:Model written to serving path /tmp/tmpbeun_83g/serving_model/taxi_simple/1589427087.
INFO:absl:Model pushed to /tmp/tmpbeun_83g/serving_model/taxi_simple/1589427087.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 17
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: /tmp/tfx-interactive-2020-05-14T01_04_27.424803-vv6id2eu/Pusher/pushed_model/17, id: 23)]
        ))

## Export to pipeline

In [73]:
_runner_type = 'beam'
_pipeline_name = 'chicage_taxi_{:s}'.format(_runner_type)

In [77]:
_notebook_filepath = os.path.join(os.getcwd(),'TFX-Estimator-Compoonent-Tutorial.ipynb')

_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')
_serving_model_dir = os.path.join(_taxi_root, 'serving_model')
_data_root = os.path.join(_taxi_root, 'data', 'simple')
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

In [75]:
components = [
    example_gen, statistics_gen, schema_gen, example_validator, transform,
    trainer, evaluator, pusher
]

In [78]:
%%skip_for_export
#docs_infra: no_execute

#@markdown Run this cell to generate the pipeline files.

if get_ipython().magics_manager.auto_magic:
    print('Warning: %automagic is ON. Line magics specified without the % prefix '
          'will not be scrubbed during export to pipeline.')

_pipeline_export_filepath = 'export_{:s}.py'.format(_pipeline_name)
context.export_to_pipeline(notebook_filepath=_notebook_filepath,
                           export_filepath=_pipeline_export_filepath,
                           runner_type=_runner_type)

INFO:absl:Exporting contents of /home/kaimo/Hands-on-TensorFlow-Extend/TFX-Estimator-Compoonent-Tutorial.ipynb to export_chicage_taxi_beam.py with beam runner.
INFO:absl:7 cell(s) marked with "%%skip_for_export", skipped.


This cell will be skipped during export to pipeline.
